In [3]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from timm import create_model
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.preprocessing import label_binarize
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler

# ==== Setup ====
data_dir = r"C:\Users\sahba\Downloads\archive (2)\IMG_CLASSES"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==== Transform ====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes
num_classes = len(class_names)

# ==== Class Weights ====
class_counts = [2697, 2215, 6946, 1827, 1644, 1888]
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = class_weights.to(device)

Using device: cuda


In [4]:
# ==== Cross Validation Setup ====
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
all_fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(full_dataset)):
    print(f"\n\n==== Fold {fold + 1}/5 ====")

    # Data Loaders
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

    # Model
    model = create_model("efficientnet_b0", pretrained=True, num_classes=num_classes)
    for param in model.parameters():
        param.requires_grad = False
    for layer in list(model.blocks)[-2:]:
        for param in layer.parameters():
            param.requires_grad = True

    in_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes)
    )
    model = model.to(device)

    # Loss, Optimizer, Scheduler
    criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
    backbone_params = [p for n, p in model.named_parameters() if not n.startswith('classifier')]
    classifier_params = model.get_classifier().parameters()
    optimizer = optim.Adam([
        {'params': backbone_params, 'lr': 1e-5},
        {'params': classifier_params, 'lr': 1e-3}
    ])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)
    scaler = GradScaler()

      # Training Loop (shortened epochs for demonstration)
    for epoch in range(20):
        print(f"\nEpoch {epoch + 1}")
        model.train()
        total, correct, running_loss = 0, 0, 0.0

        for images, labels in tqdm(train_loader, leave=False):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            running_loss += loss.item()

        print(f"Train Acc: {100 * correct / total:.2f}%, Loss: {running_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        total, correct, val_loss = 0, 0, 0.0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                val_loss += loss.item()
                total += labels.size(0)
                correct += (preds == labels).sum().item()

        val_acc = 100 * correct / total
        print(f"Val Acc: {val_acc:.2f}%, Loss: {val_loss / len(val_loader):.4f}")
        scheduler.step(val_loss)

    # Store fold metrics
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    all_fold_metrics.append(report)

# ==== Summary ====
avg_acc = np.mean([m['accuracy'] for m in all_fold_metrics])
print(f"\n✅ Average Accuracy over 5 folds: {avg_acc * 100:.2f}%")





==== Fold 1/5 ====


C:\Users\sahba\AppData\Local\Temp\ipykernel_8624\2916691845.py:40: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



Epoch 1


  0%|          | 0/430 [00:00<?, ?it/s]C:\Users\sahba\AppData\Local\Temp\ipykernel_8624\2916691845.py:51: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Acc: 72.43%, Loss: 1.0723
Val Acc: 78.42%, Loss: 1.1037

Epoch 2


...

Train Acc: 92.92%, Loss: 0.6902
Val Acc: 89.50%, Loss: 0.9349

Epoch 17


Train Acc: 93.24%, Loss: 0.6849
Val Acc: 88.25%, Loss: 0.9578

Epoch 18


Train Acc: 94.09%, Loss: 0.6644
Val Acc: 90.58%, Loss: 0.9158

Epoch 19


Train Acc: 94.08%, Loss: 0.6602
Val Acc: 89.79%, Loss: 0.9298

Epoch 20


Train Acc: 94.56%, Loss: 0.6520
Val Acc: 89.85%, Loss: 0.9332

✅ Average Accuracy over 5 folds: 89.44%
